<a href="https://colab.research.google.com/github/cryoTUD/ColabScale/blob/development/ColabScale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://gitlab.tudelft.nl/aj-lab/locscale/-/raw/master/doc/img/LocScale_logo.png" height="200" align="right" style="height:240px">

#```ColabScale```

Easy to use cryo-EM map sharpening using [```LocScale```](https://gitlab.tudelft.nl/aj-lab/locscale) and generation of feature-enhanced maps with [```LocScale-EMmerNet```](https://gitlab.tudelft.nl/aj-lab/emmernet).


For more details, see <a href="#Instructions">instructions</a> at the bottom of the notebook and read our manuscripts.


In [1]:
# @title Setup environment
#@markdown #### Please make sure to connect to a GPU runtime before starting.
#%%capture
%%time
import os

if not os.path.isfile("CONDA_READY"):
   print("installing conda...")
   os.system("pip install -q condacolab")
   import condacolab
   condacolab.install()
   os.system("conda create -n locscale python=3.11 -y")
   os.system("conda activate locscale")
   os.system("conda install -c conda-forge gfortran")
   os.system("touch CONDA_READY")

# downgrade base python to Python3.8 and install dependencies
!conda init
!conda install python=3.11 --no-pin
!pip install git+https://gitlab.tudelft.nl/aj-lab/locscale.git
!pip install stackview==0.8.0 ipycanvas==0.11 ipykernel==6.29.5 pyclesperanto_prototype -U tifffile[all]

# include side packages from 3.11
import sys
sys.path.append('/usr/local/lib/python3.11/site-packages')



installing conda...
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
modified      /root/.bashrc

==> For changes to take effect, close and re-open your current shell. <==

Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / done


==> WARNING: A newe

  ERROR: Operation cancelled by user
^C
CPU times: user 1.67 s, sys: 451 ms, total: 2.12 s
Wall time: 4min 45s


In [2]:
!conda init

no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
no change     /root/.bashrc
No action taken.


In [4]:
!conda list | grep locscale

locscale                  2.3                      pypi_0    pypi


In [10]:
from locscale.automate.tools import LocScaleInputs, LocScaleRun

locscale_inputs = LocScaleInputs()
locscale_inputs.input["halfmap_paths"] = ["/content/emd_0193_half_map_1.map", "/content/emd_0193_half_map_2.map"]
locscale_inputs.input["output_path"] = "/content/emd_0193_map.mrc"
locscale_inputs.input["gpu_ids"] = ["0"]
locscale_inputs.input["batch_size"] = 16
locscale_inputs.input["number_processes"] = 4
locscale_inputs.input["verbose"] = True
locscale_run = LocScaleRun(locscale_inputs, "test", "model_free", 4)


In [11]:
locscale_run.prepare_job()

In [12]:
locscale_run.submit_job()

In [7]:
from locscale.include.emmer.ndimage.map_utils import load_map
emmap_path = "/content/emd_0193_half_map_1.map"
emmap = load_map(emmap_path)[0]

In [8]:
emmap.shape

(230, 230, 230)

In [ ]:
#@markdown ### Input

from google.colab import files
import os

job_name = 'test' #@param {type:"string"}
job_type = "feature_enhance" # @param ["model-based", "model-free", "hybrid", "feature_enhance"]
#@markdown - __model-based__: ```LocScale``` sharpening using atomic model
#@markdown - __model-free__: ```LocScale``` sharpening without atomic model
#@markdown - __hybrid__: ```LocScale``` sharpening with partial atomic model
#@markdown - __feature_enhance__: Confidence-aware density modification with ```LocScale-EMmerNet```

upload_or_link = "upload" #@param ["upload", "link", "example"]

use_model = False #@param {type:"boolean"}
if use_model:
   if upload_or_link == "upload":
      input_model_path = os.path.join(job_name,f"model")
      os.makedirs(input_model_path, exist_ok=True)
      print("Please select model file...")
      uploaded = files.upload()
      input_model_path = os.path.join(input_model_path, list(uploaded.keys())[0])
      for model in uploaded.keys():
         if model.endswith('.pdb'):
            os.rename(model,os.path.join(input_model_path,model))
         else:
            print("Uploaded file is not a PDB file; please select correct file...")
            os.remove(model)
            uploaded = files.upload()
            os.rename(model,os.path.join(input_model_path,model))
            input_model_path = os.path.join(input_model_path, list(uploaded.keys())[0])
elif not use_model:
   input_model_path = None


#@markdown - `True` will request upload of atomic model as scaling reference. ADP refinement proceeds automaticallly.
#@markdown - `False` enables model-free scaling.

#input_model_path = "/content/map.mrc"

#uploaded_maps = []
#half_maps = {}

#use_half_maps = True #@param {type:"boolean"}
#if use_half_maps == True:
   #while len(uploaded_map) != 2:
       #if len(uploaded_map) == 0:
       #    print("Please upload two half maps.")
       #elif len(uploaded_map) == 1:
       #    print("Please upload the second half map.")
       #else:
       #    print("You have uploaded too many maps. Please upload exactly 2 maps.")
       #    uploaded_map = []
       #half_maps = files.upload()
       #uploaded_map += list(half_maps.keys())

use_half_maps = True #@param {type:"boolean"}
if use_half_maps:
  input_half_map_path = os.path.join(jobname,f"half_maps")
  os.makedirs(input_half_map_path, exist_ok=True)
  print("Please select half maps...")
  uploaded = files.upload()
  for map in uploaded.keys():
     if map.endswith('.mrc'):
        os.rename(map,os.path.join(input_half_map_path,map))
     else:
        print("Uploaded file is not a MRC file; please select correct file...")
        os.remove(map)
        uploaded = files.upload()
        os.rename(model,os.path.join(input_half_map_path,map))
elif not use_half_maps:
   input_half_map_path = None

#@markdown - `True` requires upload of unfiltered half maps (__recommended__)
#@markdown - `False` will prompt upload of full map

#@markdown #### Other options

#model_resolution = None #@param {type:"string"}
symmetry = "C1" #@param {type:"string"}

#@markdown - Specifies point group for map symmetrisation. Supported groups are C<sub>n</sub>, D<sub>n</sub>, T, O, I
#@markdown - Helical symmetry is not yet supported

output_name = None #@param {type:"string"}
#@markdown - Base string for output file names
#@markdown - `None` will use __`job_name`__

import gzip

def uncompress_if_needed(file_path):
  """Uncompresses the file at the given path if it is compressed."""
  if file_path.endswith('.gz'):
    uncompressed_path = file_path[:-3]
    with gzip.open(file_path, 'rb') as f_in:
      with open(uncompressed_path, 'wb') as f_out:
        f_out.write(f_in.read())
    print(f"Uncompressed {file_path} to {uncompressed_path}")
    return uncompressed_path
  return file_path

# Example usage for half maps:
if use_half_maps:
  input_half_map_path_1 = uncompress_if_needed(input_half_map_path_1)
  input_half_map_path_2 = uncompress_if_needed(input_half_map_path_2)
else:
  input_map_path = uncompress_if_needed(input_map_path)

# Example usage for model:
if use_model and upload_or_link == "upload":
  input_model_path = uncompress_if_needed(input_model_path)


In [ ]:
#@markdown ## Advanced Options

from google.colab import files
import os

#@markdown Most of these options should be left at default. Please only change if necessary and if you know what you are doing.

#@markdown #### FDR options

fdr_threshold = 0.01 #@param {type:"string"}
fdr_window_size = None #@param {type:"string"}
fdr_filter = None #@param {type:"string"}
averaging_filter_size = 3 #@param {type:"string"}
mask_threshold = 0.99 #@param {type:"string"}

#@markdown \

#@markdown #### EMmerNet options
model_path = None #@param {type:"string"}
low_context_model = False #@param {type:"boolean"}
batch_size = 8 #@param {type:"string"}
cube_size = 32 #@param {type:"string"}
gpu_ids = None #@param {type:"string"}

#@markdown \

#@markdown #### Reference options

model_resolution = None #@param {type:"string"}

In [ ]:
#@markdown ## Run LocScale
!locscale feature_enhance -np 8 -hm drive/MyDrive/ColabScale/emd3180/emd3180_half1.mrc drive/MyDrive/ColabScale/emd3180/emd3180_half2.mrc -v -o test.mrc -gpus 0

In [ ]:
#@title #Analyse results
#@markdown ### Display scaled and unscaled maps

#!pip install stackview==0.8.0
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
import stackview
import mrcfile
import ipywidgets as widgets
from skimage.io import imread
from google.colab import output
output.enable_custom_widget_manager()
from ipywidgets import HBox, VBox
#import pyclesperanto_prototype as cle

#cle.select_device("cupy")


export = False #@param {type:"boolean"}
#@markdown - export output as PNG images
input_colormap = "gray" #@param ['gray','plasma', 'viridis', 'inferno']
locscale_colormap = "inferno" #@param ['gray','plasma', 'viridis', 'inferno']
#@markdown - `gray`: greyscale
#@markdown - Other options are [perceptually uniform sequential color maps](https://matplotlib.org/stable/users/explain/colors/colormaps.html#sequential)
zoom_factor = 1 # @param {type:"number"}
display_style = "curtain" #@param {type:"string"}['curtain','stacked', 'toggle']

# load data
input_map = mrcfile.open('drive/MyDrive/ColabScale/input_map.mrc').data #needs to automatically load input map or averaged half maps
scaled_map = mrcfile.open('drive/MyDrive/ColabScale/scaled_map.mrc').data #needs to automatically load output map (locscale or feature_enhanced map)

# set scale
input_map = input_map/input_map.max()*255
scaled_map = scaled_map/scaled_map.max()*255

# set style & arrange widgets
if display_style == "curtain":
  print("Input map (left) vs. LocScale map (right)\n")
  w1 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=0, colormap=input_colormap, curtain_colormap=locscale_colormap)
  w2 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=1, colormap=input_colormap, curtain_colormap=locscale_colormap)
  w3 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=2, colormap=input_colormap, curtain_colormap=locscale_colormap)
  plot_map = HBox([w1, w2, w3])
elif display_style == "stacked":
  print("Input map (top) vs. LocScale map (bottom)\n")
  w1 = stackview.orthogonal(input_map,zoom_factor=zoom_factor, colormap=input_colormap)
  w2 = stackview.orthogonal(scaled_map,zoom_factor=zoom_factor, colormap=locscale_colormap)
  plot_map = VBox([w1, w2])
elif display_style == "toggle":
   print("Use buttons to toggle between maps")
   plot_map = stackview.switch(
     {"Input":    input_map,
     "LocScale": scaled_map,
     },
     colormap=[input_colormap, locscale_colormap],
     toggleable=True)
plot_map

Input map (left) vs. LocScale map (right)



In [ ]:
#@title #Analyse results
#@markdown ### Display scaled and unscaled maps

#!pip install stackview==0.8.0
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
import stackview
import mrcfile
import ipywidgets as widgets
from skimage.io import imread
from google.colab import output
output.enable_custom_widget_manager()
from ipywidgets import HBox, VBox
#import pyclesperanto_prototype as cle

#cle.select_device("cupy")


export = False #@param {type:"boolean"}
#@markdown - export output as PNG images
input_colormap = "gray" #@param ['gray','plasma', 'viridis', 'inferno']
locscale_colormap = "inferno" #@param ['gray','plasma', 'viridis', 'inferno']
#@markdown - `gray`: greyscale
#@markdown - Other options are [perceptually uniform sequential color maps](https://matplotlib.org/stable/users/explain/colors/colormaps.html#sequential)
zoom_factor = 1 # @param {type:"number"}
display_style = "stacked" #@param {type:"string"}['curtain','stacked', 'toggle']

# load data
input_map = mrcfile.open('drive/MyDrive/ColabScale/emd3180/emd3180_half1.mrc').data #needs to automatically load input map or averaged half maps
scaled_map = mrcfile.open('drive/MyDrive/ColabScale/emd3180/emd3180_feature_enhanced.mrc').data #needs to automatically load output map (locscale or feature_enhanced map)

# set scale
input_map = input_map/input_map.max()*255
scaled_map = scaled_map/scaled_map.max()*255

# set style & arrange widgets
if display_style == "curtain":
  print("Input map (left) vs. LocScale map (right)\n")
  w1 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=0, colormap=input_colormap, curtain_colormap=locscale_colormap)
  w2 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=1, colormap=input_colormap, curtain_colormap=locscale_colormap)
  w3 = stackview.curtain(input_map,scaled_map, zoom_factor=zoom_factor, axis=2, colormap=input_colormap, curtain_colormap=locscale_colormap)
  plot_map = HBox([w1, w2, w3])
elif display_style == "stacked":
  print("Input map (top) vs. LocScale map (bottom)\n")
  w1 = stackview.orthogonal(input_map,zoom_factor=zoom_factor, colormap=input_colormap)
  w2 = stackview.orthogonal(scaled_map,zoom_factor=zoom_factor, colormap=locscale_colormap)
  plot_map = VBox([w1, w2])
elif display_style == "toggle":
   print("Use buttons to toggle between maps")
   plot_map = stackview.switch(
     {"Input":    input_map,
     "LocScale": scaled_map,
     },
     colormap=[input_colormap, locscale_colormap],
     toggleable=True)
plot_map

Input map (top) vs. LocScale map (bottom)



In [ ]:
#!pip install ipyvolume
#import ipyvolume
#!jupyter nbextension enable /usr/local/lib/python3.8/site-packages/ipyvolume
#from google.colab import output
#output.enable_custom_widget_manager()
#ipyvolume.volshow(input_map)
import sys
print(sys.prefix)

/usr


In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\".


files.download(f"{jobname}.result.zip")

if save_to_google_drive == True and drive:
  uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
  uploaded.SetContentFile(f"{jobname}.result.zip")
  uploaded.Upload()
  print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")